<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Optimal Transport: An Appetizer</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

# Optimal Transport: An Appetizer
Exercises for the third lab session, see the corresponding slides on KIRO, the official *Moodle* of our university (i.e., UniPv, Italy).

### Running this notebook on Colab
Run the following snippet if you are running this notebook in [Colab](https://colab.research.google.com/).

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

## Introduction
Consider the two following 1D discrete distributions:
$$
\mu = \left[ (\mu_1=0.01, x_1=0,), (\mu_2=0.02, x_2=1), (\mu_3=0.97, x_3 = 2) \right] 
$$
$$
\nu = \left[ (\nu_1=0.3, x_1=0,), (\nu_2=0.39, x_2=1), (\nu_3=0.14, x_3 = 2), (\nu_4=0.17, x_4 = 3) \right]
$$
with cost given by the ground distance $c(x_i, x_j) = |x_i - x_j|^2$.

In [1]:
# The previous distribution are generated with the following snippet:
import numpy as np   # NumPy: Numerical Python

Normalize = lambda x: x/sum(x)

np.random.seed(13)
Mu = Normalize(np.random.chisquare(1, 4))
Nu = Normalize(np.random.chisquare(1, 3))

print(Mu)
print(Nu)

[0.30374952 0.38775918 0.14385246 0.16463884]
[0.00854421 0.02383016 0.96762564]


Note the $\mu$ and $\nu$ are [**Numpy**](https://numpy.org/) array, which are similar to **Matlab** and **C** vector:

In [2]:
print(type(Mu), type(Nu))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


Using Numpy, it is possible to define also matrix like in Matlab:

In [3]:
Zero = np.zeros((3,3))
print(Zero)

One = np.ones((4,3))
print(One)

Diag = np.zeros((5,5))
for i in range(5):
    Diag[i,i] = 1
print(Diag)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


**SIMPLE BENCHMARKING:** Sometime, it is important to measure the runtime of different algorithms that solve the same problem. To measure runtime, you can use the [perf_counter()](https://docs.python.org/3/library/time.html#time.perf_counter) function of the [time]() library:

In [ ]:
from time import perf_counter
t0 = perf_counter()
# RUN LONG TIME INTENSIVE COMPUTATION
print(perf_counter() - t0)

**SOLUTION CHECKER:** In algorithm development, it is always crucial to have a **solution checker**, independent of the solution algorithm, that is used to check that the algorithm under development always returns at least a feasible solution.

Hence, for this lab session, write a function that takes in input two discrete distributions $\mu$ and $\nu$, and a transportation plan $\pi$, and check if the plan is feasible.

In [ ]:
# TO BE COMPLETED
    def CheckSolution(Mu, Nu, plan, tol=0.000001):
    return True

**GRAPHICAL REPRESENTATION:** In addition to a solution checker, it is recommended to always try to have a graphical intuitive representation of the solution. For the transportation problem of this Lab session, you can use the following function:

In [ ]:
def MovingAvg(Ls, w=3):
    return [sum(Ls[max(0,i-w):min(len(Ls),i+w)])/(2*w) for i in range(len(Ls))]

def PlotGauss(x, mu, nu, plan):
    from matplotlib import pyplot as mp
    from math import ceil

    mp.plot(x, mu)
    mp.plot(x, nu)
    
    # Displacement Interpolation
    a = 0.5
    pi = [0.0 for _ in x]
    for i,j in plan:
        h = ceil(a*i + (1-a)*j)
        pi[h] += plan[i,j]
        
    mp.plot(x, MovingAvg(pi, w=5))
                
    #mp.savefig("twoMeasInter.pdf", bbox_inches='tight')
    mp.show()

## Exercises
In the following, try to solve the following exercise:

1. Write on paper the full LP model to compute $W_e(\mu, \nu)$ with the given ground distance. 

2. Implement in Pyomo your Model, and verify that you solution found by hand is indeed an optimal solution.

3. Write on paper the dual of the LP problem, along with the *complementary slackness conditions*. What can you deduce by the slackness conditions? 

4. Apply the **Complementary Slackness Theorem* to design an algorithm that solve 1D Optimal Transport problem with a runtime complexity of $O(n+m)$, where $m$ is the size of $\mu$ and $m$ of $\nu$. You algorithm must compute both the primal and the dual optimal solutions.

5. Compare the running time of your algorithm with the runtime of GLPK used via Pyomo, for increasing size of $n$ and $m$.

6. What can you deduce by the *complementary slackness condition* for 2D OT problems? Can you design an efficient algorithm with complexity lower than $O((n+m) \log{(n+m)})$?


**TODO:** COMPLETE BELOW WITH YOUR CODE.

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory
from pyomo.environ import maximize, minimize, Binary, RangeSet, ConstraintList

# WRITE YOUR MODEL
def OT_LP(Mu, Nu):
    # Main Pyomo model
    model = ConcreteModel()
    
    # Return the optimal solution value, and a dictionary with the optimal plan
    # dictionary: (i,j) -> \pi_ij
    return 0, []

In [ ]:
# WRITE YOUR 1D OT algorithm
def OT_1D(Mu, Nu):
    # Return the optimal solution value, and a dictionary with the optimal plan
    # dictionary: (i,j) -> \pi_ij
    return 0, []

Once you have written the two functions above, you can compare the running time:

In [ ]:
from time import perf_counter

for c in range(100):
    np.random.seed(13*c)
    Mu = Normalize(np.random.chisquare(1, 200))
    Nu = Normalize(np.random.chisquare(1, 100))

    t0 = perf_counter()
    zs, pi_LP = OT_LP(Mu, Nu)
    print(zs, perf_counter() - t0, CheckSolution(Mu, Nu, pi_LP))
    
    t0 = perf_counter()
    z, pi = OT_1D(Mu, Nu)
    print(z, perf_counter() - t0, CheckSolution(Mu, Nu, pi))

## Displacement Interpolation
If you like to play with the displacement interpolation figures of the slides, you can run the following code:

In [ ]:
x = np.linspace(0, 10, 2000)
Mu = Normalize(Gauss(x, 2, 0.4)+Gauss(x, 5, 1.2))
Nu = Normalize(Gauss(x, 6, 1))

PlotGauss(x, Mu, Nu, pi)